# Import d'un csv depuis Google Drive, nettoyage et re-export vers BigQuery.

In [1]:
import pandas as pd
import numpy as np
import os
import shapely
# libs nécessaires pour requêter BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account



# Variables à changer selon le fichier importé et la table de destination !!

In [28]:

url = 'https://drive.google.com/file/d/1dZOsOBXSI4sPjwnvzZSQfMY7P9UdzeaI/view?usp=drive_link'  # Lien du fichier sur Google Drive
dataset_name = '04bis_AirQuality_AlternativeMethod'  # Nom du dataset. S'il n'existe pas dans BigQuery, il sera créé, s'il existe, le définit comme dataset d'arrivée pour la table
table_name = '2018_AirQuality_PM10_clean_interpolated'  # Nom de la table. La crée si elle n'existe pas, ATTENTION! SI LA TABLE EXISTE DEJA, ELLE SERA ECRASEE PAR DEFAUT!

## Chargement du fichier csv depuis Google Drive

In [29]:
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

In [30]:
df.head(10)

,Unnamed: 0,PA15L:PM10,AUT:PM10,RD934:PM10,OPERA:PM10,RAMBO:PM10,BOB:PM10,GEN:PM10,LOGNES:PM10,NOGENT:PM10,...,BP_EST:PM10,TREMB:PM10,RUR_S:PM10,HAUS:PM10,BASCH:PM10,VITRY:PM10,DEF:PM10,ELYS:PM10,RN6:PM10,CERGY:PM10
0,NaN,Paris stade Lenglen,Boulevard Péripherique Auteuil,RD934 Coulommiers,Place de l'Opéra,Rambouillet,BOBIGNY,GENNEVILLIERS,LOGNES,NOGENT-SUR-MARNE,...,Boulevard Périphérique Est,TREMBLAY-EN-FRANCE,Zone Rurale Sud - Bois Herpin,Boulevard Haussmann,Place Victor Basch,VITRY-SUR-SEINE,LA DEFENSE,Avenue des Champs Elysées,Route Nationale 6 - MELUN,CERGY-PONTOISE
1,NaN,PA15L,AUT,RD934,OPERA,RAMBO,BOB,GEN,LOGNES,NOGENT,...,BP_EST,TREMB,RUR_S,HAUS,BASCH,VITRY,DEF,ELYS,RN6,CERGY
2,NaN,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,...,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules,PM 10 particules
3,NaN,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,...,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10
4,NaN,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,...,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3,microg/m3
5,2018-01-01 01:00:00Z,14.7,NaN,21.8,NaN,15.7,12.8,12.2,17.5,15.7,...,18.1,14.1,8.9,19.3,19.8,18.2,12.6,21.3,19.6,11.1
6,2018-01-01 02:00:00Z,14.1,NaN,17.3,NaN,12.2,12.6,8.6,13.5,11.4,...,14.7,12.3,9.6,20.0,14.0,10.8,10.1,30.1,16.3,8.8
7,2018-01-01 03:00:00Z,11.2,NaN,16.2,NaN,12.1,9.0,8.5,11.0,9.5,...,11.6,10.7,11.8,24.3,14.9,9.4,11.3,29.4,13.4,7.7
8,2018-01-01 04:00:00Z,12.8,NaN,14.0,NaN,14.6,6.7,8.5,13.8,11.7,...,14.4,12.2,13.6,21.4,19.9,14.0,10.2,24.6,18.5,12.9
9,2018-01-01 05:00:00Z,16.2,NaN,15.5,NaN,12.8,11.7,12.2,14.7,12.5,...,18.2,14.0,10.9,21.8,21.4,18.3,11.6,22.1,19.2,15.1


## Cleaning des rows inutiles et renaming de la première valeur du header null en "Date"

In [31]:
df.loc[1, 'Unnamed: 0'] = 'Date'
df = df.drop(df.index[[0, 2, 3, 4]]) # drop les lignes 0, 2, 3, et 4 des datasets post-2018 (lignes "inutiles")
new_header = df.iloc[0] # grab the first row for the header
df = df[1:] # take the data less the header row
df.columns = new_header # set the header row as the df header
df.head(10)

1,Date,PA15L,AUT,RD934,OPERA,RAMBO,BOB,GEN,LOGNES,NOGENT,...,BP_EST,TREMB,RUR_S,HAUS,BASCH,VITRY,DEF,ELYS,RN6,CERGY
5,2018-01-01 01:00:00Z,14.7,NaN,21.8,NaN,15.7,12.8,12.2,17.5,15.7,...,18.1,14.1,8.9,19.3,19.8,18.2,12.6,21.3,19.6,11.1
6,2018-01-01 02:00:00Z,14.1,NaN,17.3,NaN,12.2,12.6,8.6,13.5,11.4,...,14.7,12.3,9.6,20.0,14.0,10.8,10.1,30.1,16.3,8.8
7,2018-01-01 03:00:00Z,11.2,NaN,16.2,NaN,12.1,9.0,8.5,11.0,9.5,...,11.6,10.7,11.8,24.3,14.9,9.4,11.3,29.4,13.4,7.7
8,2018-01-01 04:00:00Z,12.8,NaN,14.0,NaN,14.6,6.7,8.5,13.8,11.7,...,14.4,12.2,13.6,21.4,19.9,14.0,10.2,24.6,18.5,12.9
9,2018-01-01 05:00:00Z,16.2,NaN,15.5,NaN,12.8,11.7,12.2,14.7,12.5,...,18.2,14.0,10.9,21.8,21.4,18.3,11.6,22.1,19.2,15.1
10,2018-01-01 06:00:00Z,15.4,NaN,18.8,NaN,17.9,17.4,12.4,17.2,13.5,...,16.3,14.0,10.8,26.6,24.6,15.9,14.8,23.9,17.8,17.2
11,2018-01-01 07:00:00Z,14.8,NaN,16.6,NaN,16.3,19.9,NaN,17.9,12.8,...,19.7,14.8,12.5,25.4,22.0,14.6,15.9,24.0,15.5,18.0
12,2018-01-01 08:00:00Z,12.7,NaN,20.2,NaN,20.1,15.9,NaN,18.8,14.3,...,19.9,16.9,15.9,21.5,24.0,15.3,15.9,23.4,15.4,18.9
13,2018-01-01 09:00:00Z,17.7,NaN,23.5,NaN,15.0,18.9,16.5,16.9,14.1,...,20.0,17.5,15.9,19.3,22.5,22.7,17.3,21.0,19.9,22.0
14,2018-01-01 10:00:00Z,12.7,NaN,19.4,NaN,12.9,14.8,15.5,18.2,18.5,...,18.6,15.0,9.9,18.4,18.5,18.1,18.6,19.7,15.7,14.5


In [32]:
df = df.reset_index(drop=True)
df.head(15)

1,Date,PA15L,AUT,RD934,OPERA,RAMBO,BOB,GEN,LOGNES,NOGENT,...,BP_EST,TREMB,RUR_S,HAUS,BASCH,VITRY,DEF,ELYS,RN6,CERGY
0,2018-01-01 01:00:00Z,14.7,NaN,21.8,NaN,15.7,12.8,12.2,17.5,15.7,...,18.1,14.1,8.9,19.3,19.8,18.2,12.6,21.3,19.6,11.1
1,2018-01-01 02:00:00Z,14.1,NaN,17.3,NaN,12.2,12.6,8.6,13.5,11.4,...,14.7,12.3,9.6,20.0,14.0,10.8,10.1,30.1,16.3,8.8
2,2018-01-01 03:00:00Z,11.2,NaN,16.2,NaN,12.1,9.0,8.5,11.0,9.5,...,11.6,10.7,11.8,24.3,14.9,9.4,11.3,29.4,13.4,7.7
3,2018-01-01 04:00:00Z,12.8,NaN,14.0,NaN,14.6,6.7,8.5,13.8,11.7,...,14.4,12.2,13.6,21.4,19.9,14.0,10.2,24.6,18.5,12.9
4,2018-01-01 05:00:00Z,16.2,NaN,15.5,NaN,12.8,11.7,12.2,14.7,12.5,...,18.2,14.0,10.9,21.8,21.4,18.3,11.6,22.1,19.2,15.1
5,2018-01-01 06:00:00Z,15.4,NaN,18.8,NaN,17.9,17.4,12.4,17.2,13.5,...,16.3,14.0,10.8,26.6,24.6,15.9,14.8,23.9,17.8,17.2
6,2018-01-01 07:00:00Z,14.8,NaN,16.6,NaN,16.3,19.9,NaN,17.9,12.8,...,19.7,14.8,12.5,25.4,22.0,14.6,15.9,24.0,15.5,18.0
7,2018-01-01 08:00:00Z,12.7,NaN,20.2,NaN,20.1,15.9,NaN,18.8,14.3,...,19.9,16.9,15.9,21.5,24.0,15.3,15.9,23.4,15.4,18.9
8,2018-01-01 09:00:00Z,17.7,NaN,23.5,NaN,15.0,18.9,16.5,16.9,14.1,...,20.0,17.5,15.9,19.3,22.5,22.7,17.3,21.0,19.9,22.0
9,2018-01-01 10:00:00Z,12.7,NaN,19.4,NaN,12.9,14.8,15.5,18.2,18.5,...,18.6,15.0,9.9,18.4,18.5,18.1,18.6,19.7,15.7,14.5


## Interpolation des valeurs NaN grâce à des méthodes linéaires et cubic, et résumé de l'opération.

In [33]:
# Count the number of NaN values before interpolation
nan_count_before = df.isna().sum().sum()

# Perform the interpolation (your existing code)
for col in df.columns:
    if col != 'Date':
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, set invalid values to NaN

df = df.apply(lambda col: col.interpolate(method='linear', limit=3) if col.name != 'Date' else col)

df = df.apply(
    lambda col: col.interpolate(method='cubic') if col.name != 'Date' and col.notna().sum() >= 4 else col
)

for col in df.columns:
    if col != 'Date':
        nan_indices = df[col][df[col].isna()].index
        for idx in nan_indices:
            neighbors = df.drop(columns=['Date', col]).loc[idx]
            df.at[idx, col] = neighbors.mean()

# Count the number of NaN values after interpolation
nan_count_after = df.isna().sum().sum()

# Calculate the number of interpolated values
interpolated_values = nan_count_before - nan_count_after

# Calculate the total number of values in the DataFrame (excluding the 'Date' column)
total_values = df.drop(columns=['Date']).size

# Calculate the percentage of interpolated values
percentage_interpolated = (interpolated_values / total_values) * 100

print(f"Number of interpolated values: {interpolated_values}")
print(f"Total number of values: {total_values}")
print(f"Percentage of interpolated values: {percentage_interpolated:.2f}%")

# Check the result
df.head(10)

Number of interpolated values: 18960
Total number of values: 210240
Percentage of interpolated values: 9.02%


1,Date,PA15L,AUT,RD934,OPERA,RAMBO,BOB,GEN,LOGNES,NOGENT,...,BP_EST,TREMB,RUR_S,HAUS,BASCH,VITRY,DEF,ELYS,RN6,CERGY
0,2018-01-01 01:00:00Z,14.7,16.477273,21.8,16.477273,15.7,12.8,12.200000,17.5,15.7,...,18.1,14.1,8.9,19.3,19.8,18.2,12.6,21.3,19.6,11.1
1,2018-01-01 02:00:00Z,14.1,14.004545,17.3,14.004545,12.2,12.6,8.600000,13.5,11.4,...,14.7,12.3,9.6,20.0,14.0,10.8,10.1,30.1,16.3,8.8
2,2018-01-01 03:00:00Z,11.2,12.954545,16.2,12.954545,12.1,9.0,8.500000,11.0,9.5,...,11.6,10.7,11.8,24.3,14.9,9.4,11.3,29.4,13.4,7.7
3,2018-01-01 04:00:00Z,12.8,15.295455,14.0,15.295455,14.6,6.7,8.500000,13.8,11.7,...,14.4,12.2,13.6,21.4,19.9,14.0,10.2,24.6,18.5,12.9
4,2018-01-01 05:00:00Z,16.2,16.259091,15.5,16.259091,12.8,11.7,12.200000,14.7,12.5,...,18.2,14.0,10.9,21.8,21.4,18.3,11.6,22.1,19.2,15.1
5,2018-01-01 06:00:00Z,15.4,17.690909,18.8,17.690909,17.9,17.4,12.400000,17.2,13.5,...,16.3,14.0,10.8,26.6,24.6,15.9,14.8,23.9,17.8,17.2
6,2018-01-01 07:00:00Z,14.8,17.875758,16.6,17.875758,16.3,19.9,13.766667,17.9,12.8,...,19.7,14.8,12.5,25.4,22.0,14.6,15.9,24.0,15.5,18.0
7,2018-01-01 08:00:00Z,12.7,17.951515,20.2,17.951515,20.1,15.9,15.133333,18.8,14.3,...,19.9,16.9,15.9,21.5,24.0,15.3,15.9,23.4,15.4,18.9
8,2018-01-01 09:00:00Z,17.7,18.745455,23.5,18.745455,15.0,18.9,16.500000,16.9,14.1,...,20.0,17.5,15.9,19.3,22.5,22.7,17.3,21.0,19.9,22.0
9,2018-01-01 10:00:00Z,12.7,16.772727,19.4,16.772727,12.9,14.8,15.500000,18.2,18.5,...,18.6,15.0,9.9,18.4,18.5,18.1,18.6,19.7,15.7,14.5


In [36]:
df = df.reset_index(drop=True)

## Export BigQuery

In [37]:
# Mise en place des credentials BigQuery

# définition de l'emplacement de la clé du compte de service Google
credpath = os.path.join("..", "..", "..", "..", "..", "credentials", "artefact-da53-projet-final-b60d2589fda1.json")

# définition des credentials Google en variable d'environnement en pointant vers la clé du compte de service Google
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credpath

# préparation et définition des credentials du compte de service Google en pointant vers la clé JSON et en définissant les scopes d'action : drive (on a des tables à base de Google Sheets), cloud plateform, et bigquery.
credentials = service_account.Credentials.from_service_account_file(credpath,  scopes=["https://www.googleapis.com/auth/drive","https://www.googleapis.com/auth/cloud-platform","https://www.googleapis.com/auth/bigquery"])

In [38]:
# Export BigQuery - Création d'un nouveau dataset, d'une nouvelle table, et remplissage de la table avec un dataframe Pandas

# Init the BQ client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Define the dataset id
dataset_id = f'artefact-da53-projet-final.{dataset_name}'

# Create a Dataset object
dataset = bigquery.Dataset(dataset_id)

# Create the dataset
dataset = client.create_dataset(dataset, exists_ok=True)

print(f'Dataset créé : {client.project}.{dataset.dataset_id}')

# Define the table id
table_id = f'artefact-da53-projet-final.{dataset_name}.{table_name}' # remplacer nom du dataset (si besoin) et de la table (obligatoire !) 

# Create a Table object
table = bigquery.Table(table_id)

# Create a table
table = client.create_table(table, exists_ok=True)

print(f'Table créée : {client.project}.{dataset.dataset_id}.{table.table_id}')


# envoi du dataframe vers BQ
df.to_gbq(destination_table=table_id, if_exists='replace')

Dataset créé : artefact-da53-projet-final.04bis_AirQuality_AlternativeMethod
Table créée : artefact-da53-projet-final.04bis_AirQuality_AlternativeMethod.2018_AirQuality_PM10_clean_interpolated


/tmp/ipykernel_500998/784528359.py:30: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=table_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 2198.27it/s]
